In [13]:
import numpy as np
from scipy import sparse
import itertools
from random import shuffle
from math import log
import pickle

In [14]:
test_corpus = ("""human interface computer
survey user computer system response time
eps user interface system
system human system eps
user response time
trees
graph trees
graph minors trees
graph minors survey
I like graph and stuff
I like trees and stuff
Sometimes I build a graph
Sometimes I build trees""").split("\n")

print(test_corpus)

['human interface computer', 'survey user computer system response time', 'eps user interface system', 'system human system eps', 'user response time', 'trees', 'graph trees', 'graph minors trees', 'graph minors survey', 'I like graph and stuff', 'I like trees and stuff', 'Sometimes I build a graph', 'Sometimes I build trees']


In [15]:
def build_vocab(corpus):
    """
    Build a vocabulary with word frequencies for an entire corpus.
    Returns a dictionary `w -> (index, frequency)`, mapping word strings to pairs of
    word ID and word corpus frequency.
    """

    vocab_dict = {}
    for line in corpus:
        words = line.strip().split(" ")
        for word in words:
            if word not in vocab_dict:
                vocab_dict[word] = 1
            else:
                vocab_dict[word] +=1

    word_index_count_dict = {}
    word_count = 0
    for word in vocab_dict:
        word_index_count_dict[word] = (word_count , vocab_dict[word])
        word_count = word_count + 1

    return word_index_count_dict

build_vocab(test_corpus) #인덱스랑 전체 빈도 수

{'human': (0, 2),
 'interface': (1, 2),
 'computer': (2, 2),
 'survey': (3, 2),
 'user': (4, 3),
 'system': (5, 4),
 'response': (6, 2),
 'time': (7, 2),
 'eps': (8, 2),
 'trees': (9, 5),
 'graph': (10, 5),
 'minors': (11, 2),
 'I': (12, 4),
 'like': (13, 2),
 'and': (14, 2),
 'stuff': (15, 2),
 'Sometimes': (16, 2),
 'build': (17, 2),
 'a': (18, 1)}

In [26]:
def build_cooccur(vocab, corpus, window_size=3, min_count=None):
    vocab_size = len(vocab)
    id_word = {}

    for word in vocab:
        id_word[vocab[word][0]] = word

    word_id = {id_word[id_]:id_ for id_ in id_word}


    #sparse lil_matrix is optimized to operate on matrix which mostly has zeros.    
    cooccurrences = sparse.lil_matrix((vocab_size, vocab_size),dtype=np.float64)

    for i, line in enumerate(corpus):

        senetence = line.strip().split()
        #Get the ID of words from vocab dictionary
        word_ids = [vocab[word][0] for word in senetence]
        #print word_ids

        for i, center_word_id in enumerate(word_ids):
            #Get all the left side words within the window size.    
            left_context_word_ids  = word_ids[max(0, i-window_size):i]

            #Get all the right side words within the window size. 
            right_context_word_ids = word_ids[i+1: i+window_size]

            #Now update the cooccurrence matrix for the current center word 
            #using the context words list.
            #First do for the left context part and then for right context part
            cooccurrences = update_cooccurrence_matrix(cooccurrences, left_context_word_ids, center_word_id,"left_context")
            cooccurrences = update_cooccurrence_matrix(cooccurrences, right_context_word_ids, center_word_id,"right_context")

    # Now yield our tuple sequence (dig into the LiL-matrix internals to
    # quickly iterate through all nonzero cells)
    cooccurrences_tuples = []
    for i, (row, data) in enumerate(zip(cooccurrences.rows,cooccurrences.data)):
        
        print(i, row, data)
        if min_count is not None and vocab[id_word[i]][1] < min_count:
            continue

        for data_idx, j in enumerate(row):
            if min_count is not None and vocab[id_word[j]][1] < min_count:
                continue

            cooccurrences_tuples.append((i, j, float(data[data_idx])))
            #yield i, j, data[data_idx] 

    print(cooccurrences)
    return cooccurrences_tuples

In [27]:
def update_cooccurrence_matrix(cooccurrences, context_word_ids, center_word_id, side):
    #Update cooccurrence matrix based on the distance of the context word
    #from the center word. The logic for getting the context words is:
    """
    sentence = [10, 20, 50, 60, 70]
    Let center word be 50 and window size =2
    left_context =[10,20]
    right_context = [60,70]
    Updating weights:
    left context - Since 20 is close to 50, it has to be given a weight of 1
                   Since 10 is one step far from 50, it has to be given a weight of 1/2
    right context - Since 60 is close to 50, it has to be given a weight of 1
                    Since 70 is one step far from 50, it has to be given a weight of 1/2
    Logic of updating cooccurrence: The logic is based on the distance from center word.
    But the right context word list has to be reversed to apply one single logic for both
    left and right context window. The elements at the end of the list are closer to center word.
    For instance in the left_context_list
    """

    if side == "right_context":
      context_word_ids.reverse() 

    #len of context_word_ids will be used while calculating distance
    number_of_context_words = len(context_word_ids) 

    for i in range(number_of_context_words):
        distance = number_of_context_words - i
        weight = 1 / float(distance)

        #center word will act as the row and the context word is the column
        cooccurrences[center_word_id, context_word_ids[i]] += weight

    return cooccurrences


In [28]:
vocab = build_vocab(test_corpus)

In [29]:
cooccurrences = build_cooccur(vocab, test_corpus)

0 [1, 2, 5, 8] [1.0, 0.5, 2.0, 0.5]
1 [0, 2, 4, 5, 8] [1.0, 1.0, 1.0, 1.0, 0.5]
2 [0, 1, 3, 4, 5, 6] [0.5, 1.0, 0.5, 1.0, 1.0, 0.5]
3 [2, 4, 10, 11] [0.5, 1.0, 0.5, 1.0]
4 [1, 2, 3, 5, 6, 7, 8] [1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0]
5 [0, 1, 2, 3, 4, 5, 6, 7, 8] [2.0, 1.0, 1.0, 0.3333333333333333, 1.0, 1.0, 1.0, 0.5, 1.3333333333333333]
6 [2, 4, 5, 7] [0.5, 1.3333333333333333, 1.0, 2.0]
7 [2, 4, 5, 6] [0.3333333333333333, 0.5, 0.5, 2.0]
8 [0, 1, 4, 5] [0.5, 0.5, 1.0, 1.3333333333333333]
9 [10, 11, 12, 13, 14, 15, 16, 17] [1.5, 1.0, 1.0, 1.0, 1.0, 0.5, 0.3333333333333333, 1.0]
10 [3, 9, 11, 12, 13, 14, 15, 17, 18] [0.5, 1.5, 2.0, 0.8333333333333333, 1.0, 1.0, 0.5, 0.5, 1.0]
11 [3, 9, 10] [1.0, 1.0, 2.0]
12 [9, 10, 13, 16, 17, 18] [1.0, 0.5, 2.0, 2.0, 2.0, 0.5]
13 [9, 10, 12, 14] [1.0, 1.0, 2.0, 1.0]
14 [9, 10, 12, 13, 15] [1.0, 1.0, 0.6666666666666666, 1.0, 2.0]
15 [9, 10, 13, 14] [0.5, 0.5, 0.6666666666666666, 2.0]
16 [12, 17] [2.0, 1.0]
17 [9, 10, 12, 16, 18] [1.0, 0.5, 2.0, 1.0, 1.0]
18